In [1]:
import pandas as pd
import os
from pathlib import Path

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Lasso
import numpy as np 

cwd = os.getcwd()
project_dir = Path(cwd).resolve().parents[0]
interim_data_dir = os.path.join(project_dir, 'data/interim/')
processed_data_dir = os.path.join(project_dir, 'data/processed/')
main_data = pd.read_pickle(Path(processed_data_dir) / 'main_data.pkl')

In [13]:

from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures(2)
X = main_data[['spot', 'home']] # , 'avg_win', 'pit_avg_win'
Y = main_data.loc[:, main_data.columns == 'Win']
# X = poly.fit_transform(X)

# X = main[['spot', 'home', 'avg_win', 'pit_avg_win']]
Y = main_data.loc[:, main_data.columns == 'Win']

x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.25, random_state=0)

logisticRegr = LogisticRegression(penalty='l1', solver='liblinear')
logisticRegr.fit(x_train, y_train.values.ravel())

probs = logisticRegr.predict_proba(X)
main_data['EstProb'] = probs[:,1]

selection = main_data.groupby('Date')['EstProb'].nlargest(2)

clean_selection = os.path.join(interim_data_dir, 'selection.pkl')
selection.to_pickle(clean_selection)

selection.describe()

count    3614.000000
mean        0.734384
std         0.000088
min         0.732978
25%         0.734389
50%         0.734389
75%         0.734389
max         0.734389
Name: EstProb, dtype: float64

In [14]:
main_data.loc[main_data.Date == '2019-03-20', ['Date', 'GAME_ID', 'spot', 'home', 'EstProb']]

,Date,GAME_ID,spot,home,EstProb
0,2019-03-20,OAK201903200,1,False,0.734389
24296,2019-03-20,OAK201903200,2,False,0.709171
48593,2019-03-20,OAK201903200,3,False,0.682594
72887,2019-03-20,OAK201903200,4,False,0.654771
97177,2019-03-20,OAK201903200,5,False,0.625845
121472,2019-03-20,OAK201903200,6,False,0.595992
145761,2019-03-20,OAK201903200,7,False,0.565410
170048,2019-03-20,OAK201903200,8,False,0.534322
194332,2019-03-20,OAK201903200,9,False,0.502965
218456,2019-03-20,OAK201903200,1,True,0.732978
